In [1]:
import pandas as pd
import numpy as np
import time
import glob
import sys, os
import re

In [2]:
sys.path.insert(0, os.path.abspath(".."))
from src.utils.general import busca_tipo_tesis,extraccion_encabezado,busqueda_ponentes,busqueda_fecha,busqueda_secretarios,crea_tabla
from src.utils.limpieza import limpieza_texto

### Set de Prueba

In [3]:
path, dirs, files = next(os.walk("../data_prueba"))
file_count = len(files)
print(file_count)

29


In [4]:
ruta_txt = '../data_prueba/*.txt'

In [5]:
tabla_tesis_jurisprudencia_prueba = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_reiteracion','ponente','secretario','fecha'])

t0 = time.time()
n = len(glob.glob(ruta_txt))
n_juris_reiter_prueba = 0

for i in range(n):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_prueba += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux,ponentes)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_prueba = tabla_tesis_jurisprudencia_prueba.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_prueba, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_prueba)


Tiempo total: 0.05 segundos
Total tesis leídas: 29
Total tesis de Jurisprudencia por Reiteración: 3


In [6]:
tabla_tesis_jurisprudencia_prueba.head(20)

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha
0,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,1,Jaime Raúl Oropeza García,Roberto Carlos Moreno Zamorano,19 de febrero de 2004
1,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,2,Ma. del Pilar Núñez González,José Fernández Martínez,17 de febrero de 2005
2,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,3,Manuel Rojas Fonseca,Juan Carlos Ríos López,23 de febrero de 2006
3,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,4,Jaime Raúl Oropeza García,Alejandro Ramos García,25 de mayo de 2006
4,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,5,Jaime Raúl Oropeza García,Clemente Delgado Salgado,30 de noviembre de 2006
0,159814,1a./J. 71/2014 (9a.),Primera Sala,Décima Época,"Constitucional, Penal",1,Olga Sánchez Cordero de García Villegas,Ana Carolina Cienfuegos Posada,26 de octubre de 2005
1,159814,1a./J. 71/2014 (9a.),Primera Sala,Décima Época,"Constitucional, Penal",2,Olga Sánchez Cordero de García Villegas,Constanza Tort San Román,30 de noviembre de 2005
2,159814,1a./J. 71/2014 (9a.),Primera Sala,Décima Época,"Constitucional, Penal",3,Olga Sánchez Cordero de García Villegas,Francisco Octavio Escudero Contreras,2 de julio de 2008
3,159814,1a./J. 71/2014 (9a.),Primera Sala,Décima Época,"Constitucional, Penal",4,Olga Sánchez Cordero de García Villegas,Francisco Octavio Escudero Contreras,3 de diciembre de 2008
4,159814,1a./J. 71/2014 (9a.),Primera Sala,Décima Época,"Constitucional, Penal",5,José de Jesús Gudiño Pelayo,Carmina Cortés Rodríguez,1 de septiembre de 2010


In [7]:
tabla_tesis_jurisprudencia_prueba.describe()

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha
count,15,15,15,15,15,15,15,15,15
unique,3,3,2,1,3,5,9,13,14
top,159824,X.A.T. J/12 (9a.),Tribunales Colegiados de Circuito,Décima Época,Laboral,1,Olga Sánchez Cordero de García Villegas,Francisco Octavio Escudero Contreras,2 de septiembre de 2011
freq,5,5,10,15,5,3,4,2,2


## Leyendo todas las tesis en bloques de 20,000

In [3]:
tabla_tesis_jurisprudencia_01 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_reiteracion','ponente','secretario','fecha'])
tabla_tesis_jurisprudencia_02 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_reiteracion','ponente','secretario','fecha'])
tabla_tesis_jurisprudencia_03 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_reiteracion','ponente','secretario','fecha'])
tabla_tesis_jurisprudencia_04 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_reiteracion','ponente','secretario','fecha'])
tabla_tesis_jurisprudencia_05 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_reiteracion','ponente','secretario','fecha'])
tabla_tesis_jurisprudencia_06 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_reiteracion','ponente','secretario','fecha'])
tabla_tesis_jurisprudencia_07 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_reiteracion','ponente','secretario','fecha'])
tabla_tesis_jurisprudencia_08 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_reiteracion','ponente','secretario','fecha'])
tabla_tesis_jurisprudencia_09 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_reiteracion','ponente','secretario','fecha'])
tabla_tesis_jurisprudencia_10 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_reiteracion','ponente','secretario','fecha'])
tabla_tesis_jurisprudencia_11 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_reiteracion','ponente','secretario','fecha'])
tabla_tesis_jurisprudencia_12 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_reiteracion','ponente','secretario','fecha'])
tabla_tesis_jurisprudencia_13 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_reiteracion','ponente','secretario','fecha'])

In [9]:
ruta_txt = '../data/*.txt'  #Total archivos: 260,303

**0 a 20,000**

In [4]:
ruta_txt = '../data_01/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #len(range(0,20000))
n_juris_reiter_01 = 0

for i in range(n):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_01 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux,ponentes)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_01 = tabla_tesis_jurisprudencia_01.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_01, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_01)


Van 1000 iteraciones de 20000 en 41.07 segundos
Hay 187 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 85.81 segundos
Hay 313 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 134.40 segundos
Hay 412 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 183.48 segundos
Hay 546 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 233.08 segundos
Hay 665 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 282.84 segundos
Hay 787 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 332.76 segundos
Hay 902 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 382.08 segundos
Hay 1013 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 431.41 segundos

In [5]:
tabla_tesis_jurisprudencia_01.describe()

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha
count,12095,12095,12095,12095,12095,12095,12095,12095,12095
unique,2419,2412,4,2,29,5,713,2212,2196
top,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Novena Época,Administrativa,1,José Ramón Cossío Díaz,"Georgina Laso de la Vega Romero, Sofía Verónic...",19 de junio de 2008
freq,5,10,7300,11090,2550,2419,670,449,460


In [6]:
#Convierte dataframe a .csv y .pkl
pd.DataFrame(tabla_tesis_jurisprudencia_01).to_csv('../output/tabla_tesis_jurisprudencia_01.csv', index=False, encoding='utf-8-sig')
pd.DataFrame(tabla_tesis_jurisprudencia_01).to_pickle('../output/tabla_tesis_jurisprudencia_01.pkl')

**20,000 a 40,000**

In [13]:
ruta_txt = '../data_02/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(20000,40000))
n_juris_reiter_02 = 0

for i in range(n): #for i in range(20000,40000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_02 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux,ponentes)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_02 = tabla_tesis_jurisprudencia_02.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_02, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_02)


Van 1000 iteraciones de 20000 en 42.00 segundos
Hay 111 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 84.13 segundos
Hay 243 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 126.00 segundos
Hay 358 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 167.86 segundos
Hay 470 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 209.67 segundos
Hay 603 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 251.77 segundos
Hay 718 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 294.09 segundos
Hay 839 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 335.98 segundos
Hay 943 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 378.12 segundos


In [14]:
tabla_tesis_jurisprudencia_02.describe()

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha
count,12475,12475,12475,12475,12475,12475,12475,12475,12475
unique,2494,2492,5,1,33,10,754,2346,2531
top,201624,2a./J. 10/99,Tribunales Colegiados de Circuito,Novena Época,Común,1,Mariano Azuela Güitrón,Lourdes Ferrer Mac Gregor Poisot,11 de septiembre de 2000
freq,10,10,8610,12475,2520,2494,610,202,92


In [15]:
#Convierte dataframe a .csv y .pkl
pd.DataFrame(tabla_tesis_jurisprudencia_02).to_csv('../output/tabla_tesis_jurisprudencia_02.csv', index=False, encoding='utf-8-sig')
pd.DataFrame(tabla_tesis_jurisprudencia_02).to_pickle('../output/tabla_tesis_jurisprudencia_02.pkl')

**40,000 a 60,000**

In [16]:
ruta_txt = '../data_03/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(40000,60000))
n_juris_reiter_03 = 0

for i in range(n): #for i in range(40000,60000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_03 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux,ponentes)
                
                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_03 = tabla_tesis_jurisprudencia_03.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_03, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_03)


Van 1000 iteraciones de 20000 en 42.06 segundos
Hay 119 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 83.93 segundos
Hay 239 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 125.97 segundos
Hay 354 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 168.06 segundos
Hay 478 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 210.27 segundos
Hay 593 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 252.53 segundos
Hay 707 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 294.78 segundos
Hay 840 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 336.99 segundos
Hay 946 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 379.32 segundos


In [17]:
tabla_tesis_jurisprudencia_03.describe()

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha
count,11430,11430,11430,11430,11430,11430,11430,11430,11430
unique,2286,2172,6,2,25,5,408,1424,1874
top,210733,,Tribunales Colegiados de Circuito,Octava Época,Común,1,Gustavo Calvillo Rangel,Jorge Alberto González Alvarez,15 de mayo de 1991
freq,5,30,9425,9415,3205,2286,662,275,75


In [18]:
#Convierte dataframe a .csv y .pkl
pd.DataFrame(tabla_tesis_jurisprudencia_03).to_csv('../output/tabla_tesis_jurisprudencia_03.csv', index=False, encoding='utf-8-sig')
pd.DataFrame(tabla_tesis_jurisprudencia_03).to_pickle('../output/tabla_tesis_jurisprudencia_03.pkl')

**60,000 a 80,000**

In [19]:
ruta_txt = '../data_04/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(60000,80000))
n_juris_reiter_04 = 0

for i in range(n): #for i in range(60000,80000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_04 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux,ponentes)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_04 = tabla_tesis_jurisprudencia_04.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_04, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_04)


Van 1000 iteraciones de 20000 en 42.02 segundos
Hay 89 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 83.96 segundos
Hay 150 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 125.97 segundos
Hay 227 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 168.05 segundos
Hay 294 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 210.23 segundos
Hay 379 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 252.40 segundos
Hay 429 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 294.57 segundos
Hay 471 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 336.87 segundos
Hay 514 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 379.62 segundos
H

In [20]:
tabla_tesis_jurisprudencia_04.describe()

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha
count,6399,6399,6399,6399,6399,6399,6399,6399,6399
unique,1273,701,6,2,28,10,328,868,2323
top,232480,,Tribunales Colegiados de Circuito,Octava Época,Administrativa,1,Carlos del Río Rodríguez,NA,31 de agosto de 1976
freq,10,2891,3513,3513,2136,1273,483,2258,25


In [21]:
#Convierte dataframe a .csv y .pkl
pd.DataFrame(tabla_tesis_jurisprudencia_04).to_csv('../output/tabla_tesis_jurisprudencia_04.csv', index=False, encoding='utf-8-sig')
pd.DataFrame(tabla_tesis_jurisprudencia_04).to_pickle('../output/tabla_tesis_jurisprudencia_04.pkl')

**80,000 a 100,000**

In [22]:
ruta_txt = '../data_05/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(80000,100000))
n_juris_reiter_05 = 0

for i in range(n): #for i in range(80000,100000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()
        
        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_05 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux,ponentes)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_05 = tabla_tesis_jurisprudencia_05.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_05, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_05)


Van 1000 iteraciones de 20000 en 41.70 segundos
Hay 15 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 83.09 segundos
Hay 39 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 124.45 segundos
Hay 59 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 165.86 segundos
Hay 78 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 207.43 segundos
Hay 93 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 249.15 segundos
Hay 115 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 290.90 segundos
Hay 132 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 332.74 segundos
Hay 152 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 374.58 segundos
Hay 1

In [23]:
tabla_tesis_jurisprudencia_05.describe()

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha
count,2146,2146,2146,2146,2146,2146,2146,2146,2146
unique,422,1,5,2,16,8,164,232,1334
top,242805,,Cuarta Sala,Séptima Época,Laboral,1,María Cristina Salmorán de Tamayo,NA,29 de octubre de 1970
freq,8,2146,1320,2066,1306,422,412,1233,11


In [24]:
#Convierte dataframe a .csv y .pkl
pd.DataFrame(tabla_tesis_jurisprudencia_05).to_csv('../output/tabla_tesis_jurisprudencia_05.csv', index=False, encoding='utf-8-sig')
pd.DataFrame(tabla_tesis_jurisprudencia_05).to_pickle('../output/tabla_tesis_jurisprudencia_05.pkl')

**100,000 a 120,000**

In [7]:
ruta_txt = '../data_06/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(100000,120000))
n_juris_reiter_06 = 0

for i in range(n): #for i in range(100000,120000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_06 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux,ponentes)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_06 = tabla_tesis_jurisprudencia_06.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_06, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_06)


Van 1000 iteraciones de 20000 en 44.23 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 92.71 segundos
Hay 5 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 142.49 segundos
Hay 11 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 192.48 segundos
Hay 12 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 242.64 segundos
Hay 14 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 291.61 segundos
Hay 15 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 341.96 segundos
Hay 18 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 390.34 segundos
Hay 19 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 438.44 segundos
Hay 24 tes

In [8]:
tabla_tesis_jurisprudencia_06.describe()

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha
count,135,135,135,135,135,135,135,135,135
unique,27,1,4,1,6,5,26,1,120
top,264103,,Segunda Sala,Sexta Época,Administrativa,1,Felipe Tena Ramírez,NA,29 de abril de 1958
freq,5,135,70,135,55,27,18,135,3


In [9]:
#Convierte dataframe a .csv y .pkl
pd.DataFrame(tabla_tesis_jurisprudencia_06).to_csv('../output/tabla_tesis_jurisprudencia_06.csv', index=False, encoding='utf-8-sig')
pd.DataFrame(tabla_tesis_jurisprudencia_06).to_pickle('../output/tabla_tesis_jurisprudencia_06.pkl')

**120,000 a 140,000**

In [28]:
ruta_txt = '../data_07/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(120000,140000))
n_juris_reiter_07 = 0

for i in range(n): #for i in range(120000,140000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_07 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux,ponentes)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_07 = tabla_tesis_jurisprudencia_07.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_07, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_07)



Van 1000 iteraciones de 20000 en 41.67 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 83.23 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 124.84 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 166.55 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 208.30 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 250.25 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 292.09 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 334.01 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 375.95 segundos
Hay 0 tesis de J

In [29]:
tabla_tesis_jurisprudencia_07.describe()

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha
count,0,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
#Convierte dataframe a .csv y .pkl
pd.DataFrame(tabla_tesis_jurisprudencia_07).to_csv('../output/tabla_tesis_jurisprudencia_07.csv', index=False, encoding='utf-8-sig')
pd.DataFrame(tabla_tesis_jurisprudencia_07).to_pickle('../output/tabla_tesis_jurisprudencia_07.pkl')

**140,000 a 160,000**

In [31]:
ruta_txt = '../data_08/*.txt'
    
t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(140000,160000))
n_juris_reiter_08 = 0

for i in range(n): #for i in range(140000,160000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_08 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux,ponentes)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_08 = tabla_tesis_jurisprudencia_08.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_08, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_08)



Van 1000 iteraciones de 20000 en 41.62 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 82.99 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 124.55 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 166.17 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 207.86 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 249.69 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 291.40 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 333.16 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 374.84 segundos
Hay 0 tesis de J

In [32]:
tabla_tesis_jurisprudencia_08.describe()

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha
count,0,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
#Convierte dataframe a .csv y .pkl
pd.DataFrame(tabla_tesis_jurisprudencia_08).to_csv('../output/tabla_tesis_jurisprudencia_08.csv', index=False, encoding='utf-8-sig')
pd.DataFrame(tabla_tesis_jurisprudencia_08).to_pickle('../output/tabla_tesis_jurisprudencia_08.pkl')

**160,000 a 180,000**

In [34]:
ruta_txt = '../data_09/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(160000,180000))
n_juris_reiter_09 = 0

for i in range(n): #for i in range(160000,180000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_09 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux,ponentes)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_09 = tabla_tesis_jurisprudencia_09.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_09, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_09)



Van 1000 iteraciones de 20000 en 41.50 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 82.77 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 124.27 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 165.94 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 207.58 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 249.28 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 290.89 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 332.64 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 374.30 segundos
Hay 0 tesis de J

In [35]:
tabla_tesis_jurisprudencia_09.describe()

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha
count,0,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
#Convierte dataframe a .csv y .pkl
pd.DataFrame(tabla_tesis_jurisprudencia_09).to_csv('../output/tabla_tesis_jurisprudencia_09.csv', index=False, encoding='utf-8-sig')
pd.DataFrame(tabla_tesis_jurisprudencia_09).to_pickle('../output/tabla_tesis_jurisprudencia_09.pkl')

**180,000 a 200,000**

In [37]:
ruta_txt = '../data_10/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(180000,200000))
n_juris_reiter_10 = 0

for i in range(n): #for i in range(180000,200000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_10 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux,ponentes)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_10 = tabla_tesis_jurisprudencia_10.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_10, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_10)



Van 1000 iteraciones de 20000 en 41.61 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 83.26 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 124.95 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 166.69 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 208.62 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 250.40 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 292.14 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 333.86 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 375.66 segundos
Hay 0 tesis de J

In [38]:
tabla_tesis_jurisprudencia_10.describe()

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha
count,0,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
#Convierte dataframe a .csv y .pkl
pd.DataFrame(tabla_tesis_jurisprudencia_10).to_csv('../output/tabla_tesis_jurisprudencia_10.csv', index=False, encoding='utf-8-sig')
pd.DataFrame(tabla_tesis_jurisprudencia_10).to_pickle('../output/tabla_tesis_jurisprudencia_10.pkl')

**200,000 a 220,000**

In [40]:
ruta_txt = '../data_11/*.txt'
    
t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(200000,220000))
n_juris_reiter_11 = 0

for i in range(n): #for i in range(200000,220000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_11 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux,ponentes)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_11 = tabla_tesis_jurisprudencia_11.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_11, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_11)



Van 1000 iteraciones de 20000 en 41.62 segundos
Hay 5 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 83.11 segundos
Hay 8 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 124.77 segundos
Hay 10 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 166.53 segundos
Hay 16 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 208.42 segundos
Hay 20 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 250.37 segundos
Hay 21 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 292.35 segundos
Hay 24 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 334.27 segundos
Hay 24 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 376.16 segundos
Hay 25 tes

In [41]:
tabla_tesis_jurisprudencia_11.describe()

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha
count,322,322,322,322,322,322,322,322,322
unique,50,11,6,3,11,10,61,36,158
top,388039,,Sala Auxiliar,Séptima Época,Administrativa,1,Luis Felipe Canudas Orezza,NA,5 de noviembre de 1969
freq,10,270,166,264,86,50,163,106,42


In [42]:
#Convierte dataframe a .csv y .pkl
pd.DataFrame(tabla_tesis_jurisprudencia_11).to_csv('../output/tabla_tesis_jurisprudencia_11.csv', index=False, encoding='utf-8-sig')
pd.DataFrame(tabla_tesis_jurisprudencia_11).to_pickle('../output/tabla_tesis_jurisprudencia_11.pkl')

**220,000 a 240,000**

In [43]:
ruta_txt = '../data_12/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(220000,240000))
n_juris_reiter_12 = 0

for i in range(n): #for i in range(220000,240000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_12 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux,ponentes)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_12 = tabla_tesis_jurisprudencia_12.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_12, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_12)


Van 1000 iteraciones de 20000 en 41.76 segundos
Hay 33 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 83.18 segundos
Hay 55 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 124.67 segundos
Hay 85 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 166.61 segundos
Hay 111 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 208.45 segundos
Hay 133 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 250.30 segundos
Hay 156 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 292.22 segundos
Hay 182 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 334.19 segundos
Hay 214 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 376.65 segundos
Hay

In [44]:
tabla_tesis_jurisprudencia_12.describe()

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha
count,2615,2615,2615,2615,2615,2615,2615,2615,2615
unique,520,300,7,4,29,10,292,542,1393
top,807656,,Segunda Sala,Séptima Época,Administrativa,1,Carlos del Río Rodríguez,NA,16 de febrero de 2012
freq,10,872,701,848,661,520,128,1145,43


In [45]:
#Convierte dataframe a .csv y .pkl
pd.DataFrame(tabla_tesis_jurisprudencia_12).to_csv('../output/tabla_tesis_jurisprudencia_12.csv', index=False, encoding='utf-8-sig')
pd.DataFrame(tabla_tesis_jurisprudencia_12).to_pickle('../output/tabla_tesis_jurisprudencia_12.pkl')

**240,000 a 260,303**

In [46]:
ruta_txt = '../data_13/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(240000,260303))
n_juris_reiter_13 = 0

for i in range(n): #for i in range(240000,260303):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_13 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux,ponentes)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_13 = tabla_tesis_jurisprudencia_13.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_13, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_13)



Van 1000 iteraciones de 20305 en 42.43 segundos
Hay 101 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20305 en 84.60 segundos
Hay 184 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20305 en 127.05 segundos
Hay 262 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20305 en 169.80 segundos
Hay 347 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20305 en 212.70 segundos
Hay 440 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20305 en 255.75 segundos
Hay 518 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20305 en 298.62 segundos
Hay 582 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20305 en 341.35 segundos
Hay 650 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20305 en 384.27 segundos


In [47]:
tabla_tesis_jurisprudencia_13.describe()

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha
count,8940,8940,8940,8940,8940,8940,8940,8940,8940
unique,1788,1779,4,2,40,5,704,1915,1480
top,2015062,1a./J. 46/2015 (10a.),Tribunales Colegiados de Circuito,Décima Época,Común,1,José Fernando Franco González Salas,"Fanuel Martínez López, Claudia Mendoza Polanco...",5 de octubre de 2016
freq,5,10,5255,8870,1960,1788,510,150,155


In [48]:
#Convierte dataframe a .csv y .pkl
pd.DataFrame(tabla_tesis_jurisprudencia_13).to_csv('../output/tabla_tesis_jurisprudencia_13.csv', index=False, encoding='utf-8-sig')
pd.DataFrame(tabla_tesis_jurisprudencia_13).to_pickle('../output/tabla_tesis_jurisprudencia_13.pkl')

In [49]:
n_total_juris_reiter = n_juris_reiter_01+n_juris_reiter_02+n_juris_reiter_03+n_juris_reiter_04+n_juris_reiter_05+n_juris_reiter_06+n_juris_reiter_07+n_juris_reiter_08+n_juris_reiter_09+n_juris_reiter_10+n_juris_reiter_11+n_juris_reiter_12+n_juris_reiter_13
print("Gran total tesis de Jurisprudencia por Reiteración:", n_total_juris_reiter)

Gran total tesis de Jurisprudencia por Reiteración: 11279


## Uniendo dataframes en uno solo

In [3]:
tabla_tesis_jurisprudencia_01 = pd.read_pickle('../output/tabla_tesis_jurisprudencia_01.pkl')
tabla_tesis_jurisprudencia_02 = pd.read_pickle('../output/tabla_tesis_jurisprudencia_02.pkl')
tabla_tesis_jurisprudencia_03 = pd.read_pickle('../output/tabla_tesis_jurisprudencia_03.pkl')
tabla_tesis_jurisprudencia_04 = pd.read_pickle('../output/tabla_tesis_jurisprudencia_04.pkl')
tabla_tesis_jurisprudencia_05 = pd.read_pickle('../output/tabla_tesis_jurisprudencia_05.pkl')
tabla_tesis_jurisprudencia_06 = pd.read_pickle('../output/tabla_tesis_jurisprudencia_06.pkl')
tabla_tesis_jurisprudencia_07 = pd.read_pickle('../output/tabla_tesis_jurisprudencia_07.pkl')
tabla_tesis_jurisprudencia_08 = pd.read_pickle('../output/tabla_tesis_jurisprudencia_08.pkl')
tabla_tesis_jurisprudencia_09 = pd.read_pickle('../output/tabla_tesis_jurisprudencia_09.pkl')
tabla_tesis_jurisprudencia_10 = pd.read_pickle('../output/tabla_tesis_jurisprudencia_10.pkl')
tabla_tesis_jurisprudencia_11 = pd.read_pickle('../output/tabla_tesis_jurisprudencia_11.pkl')
tabla_tesis_jurisprudencia_12 = pd.read_pickle('../output/tabla_tesis_jurisprudencia_12.pkl')
tabla_tesis_jurisprudencia_13 = pd.read_pickle('../output/tabla_tesis_jurisprudencia_13.pkl')

In [4]:
tesis_juris_raw = pd.concat([tabla_tesis_jurisprudencia_01,tabla_tesis_jurisprudencia_02,tabla_tesis_jurisprudencia_03,tabla_tesis_jurisprudencia_04,tabla_tesis_jurisprudencia_05,tabla_tesis_jurisprudencia_06,tabla_tesis_jurisprudencia_07,tabla_tesis_jurisprudencia_08,tabla_tesis_jurisprudencia_09,tabla_tesis_jurisprudencia_10,tabla_tesis_jurisprudencia_11,tabla_tesis_jurisprudencia_12,tabla_tesis_jurisprudencia_13])

In [5]:
#Convierte dataframe a .csv y .pkl
pd.DataFrame(tesis_juris_raw).to_csv('../output/tesis_juris_raw.csv', index=False, encoding='utf-8-sig')
pd.DataFrame(tesis_juris_raw).to_pickle('../output/tesis_juris_raw.pkl')

In [6]:
tesis_juris_raw.describe()

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha
count,56557,56557,56557,56557,56557,56557,56557,56557,56557
unique,11278,9150,7,6,59,10,1728,5917,9227
top,388039,,Tribunales Colegiados de Circuito,Novena Época,Común,1,Gustavo Calvillo Rangel,NA,19 de junio de 2008
freq,10,6344,35414,25580,11743,11279,1510,5088,461


## Limpieza de datos

In [7]:
import dateparser

In [8]:
tesis_juris_raw = pd.read_pickle('../output/tesis_juris_raw.pkl')

In [9]:
tesis_juris_raw = tesis_juris_raw.rename(columns={'fecha':'fecha_texto'})

In [10]:
%%capture output
tesis_juris_raw['fecha'] = tesis_juris_raw['fecha_texto'].apply(dateparser.parse)

In [11]:
tesis_juris_raw.dtypes

id_tesis                 object
tesis                    object
instancia                object
epoca                    object
materia                  object
#_reiteracion            object
ponente                  object
secretario               object
fecha_texto              object
fecha            datetime64[ns]
dtype: object

In [12]:
tesis_juris = pd.DataFrame(tesis_juris_raw.sort_values(['id_tesis','fecha']).groupby('id_tesis').apply(lambda x: x))

In [13]:
orden_fecha = tesis_juris.sort_values(['id_tesis','fecha']).groupby('id_tesis').cumcount()+1

In [14]:
tesis_juris['orden_fecha'] = list(orden_fecha)

In [15]:
tesis_juris = tesis_juris.drop(columns=['#_reiteracion'])
tesis_juris = tesis_juris.rename(columns={'orden_fecha':'#_reiteracion'})
tesis_juris = tesis_juris[['id_tesis','tesis','instancia','epoca','materia','#_reiteracion','ponente','secretario','fecha_texto','fecha']]

In [16]:
tesis_juris

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha_texto,fecha
0,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,1,Jaime Raúl Oropeza García,Roberto Carlos Moreno Zamorano,19 de febrero de 2004,2004-02-19
1,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,2,Ma. del Pilar Núñez González,José Fernández Martínez,17 de febrero de 2005,2005-02-17
2,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,3,Manuel Rojas Fonseca,Juan Carlos Ríos López,23 de febrero de 2006,2006-02-23
3,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,4,Jaime Raúl Oropeza García,Alejandro Ramos García,25 de mayo de 2006,2006-05-25
4,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,5,Jaime Raúl Oropeza García,Clemente Delgado Salgado,30 de noviembre de 2006,2006-11-30
...,...,...,...,...,...,...,...,...,...,...
0,820269,VI. 2o. 155,Tribunales Colegiados de Circuito,Octava Época,Penal,1,José Galván Rojas,Jorge Patlán Origel,2 de junio de 1988,1988-06-02
1,820269,VI. 2o. 155,Tribunales Colegiados de Circuito,Octava Época,Penal,2,Arnoldo Nájera Virgen,Enrique Crispín Campos Ramírez,11 de abril de 1989,1989-04-11
2,820269,VI. 2o. 155,Tribunales Colegiados de Circuito,Octava Época,Penal,3,Arnoldo Nájera Virgen,Nelson Loranca Ventura,26 de mayo de 1989,1989-05-26
3,820269,VI. 2o. 155,Tribunales Colegiados de Circuito,Octava Época,Penal,4,Gustavo Calvillo Rangel,Humberto Schettino Reyna,3 de abril de 1990,1990-04-03


#### Limpiando "materia"

In [17]:
pd.DataFrame(tesis_juris['materia'].str.len().value_counts()).reset_index().\
        rename(columns={'index':'len_materia','materia':'conteo'}).sort_values(by=['len_materia'],ascending=True)

,len_materia,conteo
0,5,23771
2,7,9758
5,12,1045
1,14,11936
4,21,2731
6,23,996
9,26,30
8,28,40
3,30,6105
13,35,5


In [18]:
tesis_juris.loc[tesis_juris['materia'].str.len() == 51]

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha_texto,fecha
0,160952,XI.5o.(III Región) J/2 (9a.),Tribunales Colegiados de Circuito,Décima Época,"Constitucional, Administrativa Tipo: Jurisprud...",1,Jaime Uriel Torres Hernández,Sonia Suárez Ríos,2 de junio de 2011,2011-06-02
1,160952,XI.5o.(III Región) J/2 (9a.),Tribunales Colegiados de Circuito,Décima Época,"Constitucional, Administrativa Tipo: Jurisprud...",2,Antonio Ceja Ochoa,Isaura Romero Mena,2 de junio de 2011,2011-06-02
2,160952,XI.5o.(III Región) J/2 (9a.),Tribunales Colegiados de Circuito,Décima Época,"Constitucional, Administrativa Tipo: Jurisprud...",3,Antonio Ceja Ochoa,Isaura Romero Mena,2 de junio de 2011,2011-06-02
3,160952,XI.5o.(III Región) J/2 (9a.),Tribunales Colegiados de Circuito,Décima Época,"Constitucional, Administrativa Tipo: Jurisprud...",4,Antonio Ceja Ochoa,David Israel Domínguez,9 de junio de 2011,2011-06-09
4,160952,XI.5o.(III Región) J/2 (9a.),Tribunales Colegiados de Circuito,Décima Época,"Constitucional, Administrativa Tipo: Jurisprud...",5,Jaime Uriel Torres Hernández,Sonia Suárez Ríos,16 de junio de 2011,2011-06-16
0,2003734,X.1o.(XI Región) J/1 (10a.),Tribunales Colegiados de Circuito,Décima Época,"Constitucional, Administrativa Tipo: Jurisprud...",1,Jerónimo José Martínez,Dulce María Rodríguez Terrazas,4 de enero de 2013,2013-01-04
1,2003734,X.1o.(XI Región) J/1 (10a.),Tribunales Colegiados de Circuito,Décima Época,"Constitucional, Administrativa Tipo: Jurisprud...",2,Juan Carlos Moreno Correa,Eduardo Antonio Méndez Granado,7 de febrero de 2013,2013-02-07
2,2003734,X.1o.(XI Región) J/1 (10a.),Tribunales Colegiados de Circuito,Décima Época,"Constitucional, Administrativa Tipo: Jurisprud...",3,Vicente Mariche de la Garza,Juan Carlos Méndez Moreno,13 de febrero de 2013,2013-02-13
3,2003734,X.1o.(XI Región) J/1 (10a.),Tribunales Colegiados de Circuito,Décima Época,"Constitucional, Administrativa Tipo: Jurisprud...",4,Vicente Mariche de la Garza,Juan Carlos Méndez Moreno,13 de febrero de 2013,2013-02-13
4,2003734,X.1o.(XI Región) J/1 (10a.),Tribunales Colegiados de Circuito,Décima Época,"Constitucional, Administrativa Tipo: Jurisprud...",5,Omar Gómez Brito,Dulce María Rodríguez Terrazas,6 de marzo de 2013,2013-03-06


In [19]:
tesis_juris['materia'] = tesis_juris['materia'].str.replace(' Tipo: Jurisprudencia', '')

In [20]:
pd.DataFrame(tesis_juris['materia'].str.len().value_counts()).reset_index().\
        rename(columns={'index':'len_materia','materia':'conteo'}).sort_values(by=['len_materia'],ascending=True)


,len_materia,conteo
0,5,23801
2,7,9768
5,12,1045
1,14,11941
4,21,2741
6,23,996
8,28,30
3,30,6115
7,37,95
9,39,25


#### Limpiando "tesis"

In [21]:
(tesis_juris['tesis'].values == '').sum() 

6344

In [22]:
tesis_juris.loc[tesis_juris['tesis'].str.len() == 0]

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha_texto,fecha
0,206527,,Segunda Sala,Octava Época,Común,1,Atanasio González Martínez,Guillermo David Vázquez Ortiz,25 de abril de 1988,1988-04-25
1,206527,,Segunda Sala,Octava Época,Común,2,Atanasio González Martínez,Amanda R. García González,25 de mayo de 1988,1988-05-25
2,206527,,Segunda Sala,Octava Época,Común,3,Atanasio González Martínez,Amanda R. García González,25 de mayo de 1988,1988-05-25
3,206527,,Segunda Sala,Octava Época,Común,4,Atanasio González Martínez,Guillermo David Vázquez Ortiz,01 de junio de 1988,1988-06-01
4,206527,,Segunda Sala,Octava Época,Común,5,Manuel Gutiérrez de Velasco,Guillermo David Vázquez Ortiz,22 de junio de 1988,1988-06-22
...,...,...,...,...,...,...,...,...,...,...
0,818839,,Cuarta Sala,Séptima Época,Laboral,1,María Cristina Salmorán de Tamayo,Víctor Ceja Villaseñor,13 de agosto de 1979,1979-08-13
1,818839,,Cuarta Sala,Séptima Época,Laboral,2,María Cristina Salmorán de Tamayo,J. Tomás Garrido Muñoz,29 de octubre de 1981,1981-10-29
2,818839,,Cuarta Sala,Séptima Época,Laboral,3,María Cristina Salmorán de Tamayo,J. Tomás Garrido Muñoz,8 de febrero de 1982,1982-02-08
3,818839,,Cuarta Sala,Séptima Época,Laboral,4,María Cristina Salmorán de Tamayo,J. Tomás Garrido Muñoz,01 de marzo de 1982,1982-03-01


In [23]:
tesis_juris['tesis'] = np.where(tesis_juris['tesis'].str.len() == 0, 'NA', tesis_juris['tesis'])

In [24]:
(tesis_juris['tesis'].values == '').sum()

0

#### Guardando df limpio y ordenado

In [25]:
#Convierte dataframe a .csv y .pkl
pd.DataFrame(tesis_juris).to_csv('../output/tesis_juris.csv', index=False, encoding='utf-8-sig')
pd.DataFrame(tesis_juris).to_pickle('../output/tesis_juris.pkl')

## Agregando indicador a Ponentes y Secretarios repetidos por tesis

In [26]:
tesis_juris_rep = pd.read_pickle('../output/tesis_juris.pkl')

In [27]:
tesis_juris_rep['ponente_rep'] = np.where(tesis_juris_rep.duplicated(['id_tesis','ponente'], keep='first'), tesis_juris_rep['ponente']+'_REP', tesis_juris_rep['ponente'])
tesis_juris_rep['secretario_rep'] = np.where(tesis_juris_rep.duplicated(['id_tesis','secretario'], keep='first'), tesis_juris_rep['secretario']+'_REP', tesis_juris_rep['secretario'])

In [28]:
tesis_juris_rep

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha_texto,fecha,ponente_rep,secretario_rep
0,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,1,Jaime Raúl Oropeza García,Roberto Carlos Moreno Zamorano,19 de febrero de 2004,2004-02-19,Jaime Raúl Oropeza García,Roberto Carlos Moreno Zamorano
1,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,2,Ma. del Pilar Núñez González,José Fernández Martínez,17 de febrero de 2005,2005-02-17,Ma. del Pilar Núñez González,José Fernández Martínez
2,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,3,Manuel Rojas Fonseca,Juan Carlos Ríos López,23 de febrero de 2006,2006-02-23,Manuel Rojas Fonseca,Juan Carlos Ríos López
3,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,4,Jaime Raúl Oropeza García,Alejandro Ramos García,25 de mayo de 2006,2006-05-25,Jaime Raúl Oropeza García_REP,Alejandro Ramos García
4,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,5,Jaime Raúl Oropeza García,Clemente Delgado Salgado,30 de noviembre de 2006,2006-11-30,Jaime Raúl Oropeza García_REP,Clemente Delgado Salgado
...,...,...,...,...,...,...,...,...,...,...,...,...
0,820269,VI. 2o. 155,Tribunales Colegiados de Circuito,Octava Época,Penal,1,José Galván Rojas,Jorge Patlán Origel,2 de junio de 1988,1988-06-02,José Galván Rojas,Jorge Patlán Origel
1,820269,VI. 2o. 155,Tribunales Colegiados de Circuito,Octava Época,Penal,2,Arnoldo Nájera Virgen,Enrique Crispín Campos Ramírez,11 de abril de 1989,1989-04-11,Arnoldo Nájera Virgen,Enrique Crispín Campos Ramírez
2,820269,VI. 2o. 155,Tribunales Colegiados de Circuito,Octava Época,Penal,3,Arnoldo Nájera Virgen,Nelson Loranca Ventura,26 de mayo de 1989,1989-05-26,Arnoldo Nájera Virgen_REP,Nelson Loranca Ventura
3,820269,VI. 2o. 155,Tribunales Colegiados de Circuito,Octava Época,Penal,4,Gustavo Calvillo Rangel,Humberto Schettino Reyna,3 de abril de 1990,1990-04-03,Gustavo Calvillo Rangel,Humberto Schettino Reyna


In [29]:
#Convierte dataframe a .csv y .pkl
tesis_juris_rep.to_csv('../output/tesis_juris_rep.csv', index=False, encoding='utf-8-sig')
tesis_juris_rep.to_pickle('../output/tesis_juris_rep.pkl')

### Debug individual por tesis

In [53]:
def funcion_auxiliar_ponentes(x0,x1,x2,x3):
    if len(x0)!= 0 :
        result = x0
    elif len(x1)!= 0:
        result = x1
    elif len(x2)!= 0:
        result =["No se menciona el nombre del ponente sólo del relator"]
    elif x3 == 1:
        result =["Revisar texto tesis"]    
    return result


def funcion_auxiliar_secretarios(x0,x1,x10): 
    if len(x0)!= 0 :
        result = x0
    elif len(x1)!= 0:
        result = ["NA"]
    elif x10 == 1:
        result =['NA']    
    return result

In [54]:
tesis_a_revisar = '815972'

In [55]:
dir_text = "../data_12/Tesis"+tesis_a_revisar+".txt"

file = open(dir_text,'r', encoding='utf8')
text = file.read()
file.close()

#Eliminar tabulaciones adicionales
text_aux = text.strip()

pos_tipo = text_aux.find('Tipo:')
if text_aux[pos_tipo:].find('\n\n') < text_aux[pos_tipo:].find('\n') :
    pos_final_tipo = text_aux[pos_tipo:].find('\n\n')
else:
    pos_final_tipo = text_aux[pos_tipo:].find('\n')-2
len_tipo = len('Tipo:')
tipo = text_aux[pos_tipo+len_tipo:pos_tipo+pos_final_tipo+2].replace('\n',' ').strip()

if tipo == "Jurisprudencia":
        
    if len(re.findall('Ponente',text_aux)) >= 5:
        
        #Eliminar tabulaciones adicionales
        text_aux = text.strip()
        text_aux = text_aux.replace(' Fuente:','\n\nFuente:')

        #Posicion en el texto de las variables
        pos_registro_digital = text_aux.find('Registro digital:')
        pos_final_registro_digital = text_aux[pos_registro_digital:].find('\n')

        pos_instancia = text_aux.find('Instancia:')
        pos_final_instancia_aux1 = text_aux[pos_instancia:].find('\n\n')

        lista  = []

        lista.append(text_aux[pos_instancia:].find('Primera Época'))
        lista.append(text_aux[pos_instancia:].find('Segunda Época'))
        lista.append(text_aux[pos_instancia:].find('Tercera Época'))
        lista.append(text_aux[pos_instancia:].find('Cuarta Época'))
        lista.append(text_aux[pos_instancia:].find('Quinta Época'))
        lista.append(text_aux[pos_instancia:].find('Sexta Época'))
        lista.append(text_aux[pos_instancia:].find('Séptima Época'))
        lista.append(text_aux[pos_instancia:].find('Octava Época'))
        lista.append(text_aux[pos_instancia:].find('Novena Época'))
        lista.append(text_aux[pos_instancia:].find('Décima Época'))
        lista.append(text_aux[pos_instancia:].find('Undécima Época'))

        pos_final_instancia_aux2 = min([x for x in lista if x > -1])

        if pos_final_instancia_aux1 < pos_final_instancia_aux2:
            pos_final_instancia = pos_final_instancia_aux1
        else:
            pos_final_instancia = pos_final_instancia_aux2

        pos_epoca_aux1 = text_aux.find('\n\n')

        lista  = []

        lista.append(text_aux.find('Primera Época'))
        lista.append(text_aux.find('Segunda Época'))
        lista.append(text_aux.find('Tercera Época'))
        lista.append(text_aux.find('Cuarta Época'))
        lista.append(text_aux.find('Quinta Época'))
        lista.append(text_aux.find('Sexta Época'))
        lista.append(text_aux.find('Séptima Época'))
        lista.append(text_aux.find('Octava Época'))
        lista.append(text_aux.find('Novena Época'))
        lista.append(text_aux.find('Décima Época'))
        lista.append(text_aux.find('Undécima Época'))

        pos_epoca_aux2 = min([x for x in lista if x > -1])

        if pos_epoca_aux1 < pos_epoca_aux2:
            pos_epoca = pos_epoca_aux1
        else:
            pos_epoca = pos_epoca_aux2

        if pos_epoca_aux1 < pos_epoca_aux2:
            pos_final_epoca = text_aux[pos_epoca+2:].find('\n\n')
        else:
            pos_final_epoca = text_aux[pos_epoca+2:].find('\n')

        pos_materia = text_aux.find('Materia(s):')
        pos_final_materia = text_aux[pos_materia:].find('\n\n')

        pos_tesis = text_aux.find('Tesis:')
        pos_final_tesis = text_aux[pos_tesis:].find('\n\n')

        #Tamaño de las variables
        len_registro_digital = len('Registro digital:')
        len_instancia = len('Instancia:')
        if pos_epoca_aux1 < pos_epoca_aux2:
            len_epoca = len('\n\n')
        else:
            len_epoca = 0
        len_materia = len('Materia(s):')
        len_tesis = len('Tesis:')

        tesis = text_aux[pos_tesis+len_tesis:pos_tesis+pos_final_tesis+2].replace('\n',' ').strip()
        instancia = text_aux[pos_instancia+len_instancia:pos_instancia+pos_final_instancia].replace('\n',' ').strip()
        epoca = text_aux[pos_epoca+len_epoca:pos_epoca+pos_final_epoca+2].replace('\n',' ').strip()
        materia = text_aux[pos_materia+len_materia:pos_materia+pos_final_materia+2].replace('\n',' ').strip()
        
        #Búsqueda de Id
        encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
        id_tesis = [encontrar_id_tesis.group(1)]

        #Limpieza Texto
        tesis_borrar = ''.join(id_tesis)
        text_aux = text_aux.strip()
        text_aux = text_aux.replace('\n\n','###')
        text_aux = text_aux.replace('\n',' ')
        text_aux = text_aux.replace('Ley de Amparo','ley_de_amparo')
        text_aux = text_aux.replace('Competencia Económica','competencia_económica')
        text_aux = text_aux.replace('Fomento a la Competencia','Fomento a la competencia')
        text_aux = text_aux.replace('Comisión Federal de Competencia','Comisión Federal de competencia')
        text_aux = text_aux.replace('Tribunal de Juicio Oral','Tribunal de juicio oral')
        text_aux = text_aux.replace('Amparos colectivos','amparos colectivos')
        text_aux = text_aux.replace('del año','de')
        text_aux = text_aux.replace('Presidente Ponente','Presidente-ponente')
        text_aux = text_aux.replace('al Ministro Ponente,','al Ministro-ponente')
        text_aux = text_aux.replace('autorizados por el Secretario','autorizados por el secr.')
        text_aux = text_aux.replace('Ponente: La publicación no menciona el nombre del ponente.','La publicación no menciona el nombre del ponente.')
        text_aux = text_aux.replace('de la ponencia del Ministro Jorge Mario Pardo Rebolledo (Secretario: Ricardo Antonio Silva Díaz)','de la ponencia del Ministro Jorge Mario Pardo Rebolledo (secr.: Ricardo Antonio Silva Díaz)')
        text_aux = text_aux.replace('Ponente: Pedro Guerrero Martínez. Juan Díaz Romero','Ponente: Pedro Guerrero Martínez. Secretario: Juan Díaz Romero')
        text_aux = text_aux.replace('J.Tomás Garrido Muñoz','J. Tomás Garrido Muñoz')
        text_aux = text_aux.replace('Secretaria: Secretaria: Alejandra','Secretaria: Alejandra')
        text_aux = text_aux.replace('César De La Rosa Zubran','César de la Rosa Zubran')
        text_aux = text_aux.replace('Hechos: Varios sindicatos','Hechos: varios sindicatos')
        text_aux = text_aux.replace(' en sesión de 5 de marzo de 1996','')
        text_aux = text_aux.replace('29 se septiembre de 1993','29 de septiembre de 1993')
        text_aux = text_aux.replace('12 se agosto de 1976','12 de agosto de 1976')
        text_aux = text_aux.replace('10 de 1988','10 de FALTAMES de 1988')
        text_aux = text_aux.replace('* de abril de 1969','99 de abril de 1969')
        text_aux = text_aux.replace('7 de agosto de 1984. 7 de agosto de 1984.','7 de agosto de 1984.')
        text_aux = text_aux.replace('4 de julio de 975','4 de julio de 1975')
        text_aux = text_aux.replace('sepmtiembre','septiembre')
        text_aux = text_aux.replace(r'1o.','01')
        text_aux = text_aux.replace(r'1°','01')
        text_aux = text_aux.replace(r'1º','01')
        text_aux = text_aux.replace('Pág. 1 de 1 ','')
        text_aux = text_aux.replace('Pág. 1 de 2 ','')
        text_aux = text_aux.replace('Pág. 2 de 2 ','')
        text_aux = text_aux.replace('Pág. 1 de 3 ','')
        text_aux = text_aux.replace('Pág. 2 de 3 ','')
        text_aux = text_aux.replace('Pág. 3 de 3 ','')
        text_aux = text_aux.replace('Pág. 1 de 4 ','')
        text_aux = text_aux.replace('Pág. 2 de 4 ','')
        text_aux = text_aux.replace('Pág. 3 de 4 ','')
        text_aux = text_aux.replace('Pág. 4 de 4 ','')
        text_aux = text_aux.replace('Pág. 1 de 5 ','')
        text_aux = text_aux.replace('Pág. 2 de 5 ','')
        text_aux = text_aux.replace('Pág. 3 de 5 ','')
        text_aux = text_aux.replace('Pág. 4 de 5 ','')
        text_aux = text_aux.replace('Pág. 5 de 5 ','')
        text_aux = text_aux.replace('Pág. 1 de 6 ','')
        text_aux = text_aux.replace('Pág. 2 de 6 ','')
        text_aux = text_aux.replace('Pág. 3 de 6 ','')
        text_aux = text_aux.replace('Pág. 4 de 6 ','')
        text_aux = text_aux.replace('Pág. 5 de 6 ','')
        text_aux = text_aux.replace('Pág. 6 de 6 ','')
        text_aux = text_aux.replace('http://sjf2.scjn.gob.mx/detalle/tesis/','')
        text_aux = text_aux.replace(tesis_borrar,'')
        text_aux = text_aux.replace('  Fecha de impresión','')
        text_aux = text_aux.replace('Fecha de impresión','')
        text_aux = text_aux.replace(' 02/07/2021','')
        text_aux = text_aux.replace(' 04/07/2021','')
        text_aux = text_aux.replace(' 06/07/2021','')
        text_aux = text_aux.replace(' 07/07/2021','')
        text_aux = text_aux.replace(' 08/07/2021','')
        text_aux = text_aux.replace(' 09/07/2021','')
        text_aux = text_aux.replace(' 10/07/2021','')
        text_aux = text_aux.replace('  \x0c','')
        text_aux = text_aux.replace('######\x0c','')
        text_aux = text_aux.replace('.}','.')
        text_aux = text_aux.replace('Semanario Judicial de la Federación ','')
        text_aux = re.sub(r'Se formó jurisprudencia aunque no.*',"",text_aux)
        text_aux = text_aux.replace(r'(DA. 903/88. Impulsora Industrial de Ingeniería, S. A. de C. V. 15 de junio de 1988. Unanimidad de votos. Ponente: Genaro David Góngora Pimentel. Secretaria: Adriana Leticia Campuzano Gallegos)','')

        #Búsqueda de párrafo donde se mencione ponente con fecha
        #ponente_fecha_aux = re.findall(r'(?:Amparo|Revisión|Contradicción|Queja|Facultad|Incidente|Reclamación|Inconformidad|Competencia|Recurso de|Impedimento|Acción de inconstitucionalidad)([\s\S]*?)(Ponente: |Ponente )([\s\S]*?)(Secretari\w: |Secretari\ws:|Secretari\w. )([\s\S]*?)(?:\.?)', text_aux)
        ponente_fecha_aux = re.findall(r'(?:Amparo|Revisión|Contradicción|Queja|Facultad|Incidente|Reclamación|Inconformidad|Competencia|Recurso de|Impedimento|Acción de inconstitucionalidad|Varios |Controversia constitucional|Reconocimiento de inocencia|Improcedencia|Aclaración de sentencia en la contra|Repetición|Inejecuci|Expediente vario|Solicitud|Acumulaci\wn|Trámite|Consulta|Recurso en|Juicio|Reposici\wn|Incompetencia|Incidente|Conflicto competencial|Vol\wmen \d{1,2}, p\wgina|Indulto necesario|C\. C\. \d{2,3}/\d{2})([\s\S]*?)(Ponente: |Ponente |Ponente\.|Ponente:\.|Ponente:|La publicación no menciona el nombre del ponente)([\s\S]*?)(?:Secretari\w: |Secretari\ws:|Secretari\w. )?([\s\S]*?)(?:\.?)', text_aux)
        ponente_fecha_texto = ''.join([''.join(j) for j in ponente_fecha_aux])
        
        #Búsqueda de Ponente
        #x0_ponente = re.findall("Ponente:([\s\S]*?)(?:.###|. Secretari|.Secretari|,|.\nSecretari|;|. Nota:|Pág.|Vol|.Tom|. Véase|. Amparo|. Séptima Época|. Sexta Época)",ponente_fecha_texto) 
        x0_ponente = re.findall("(?:Ponente: Secretario Lic\. |Ponente: |Ponente |Ponente. |Ponentes: |Ponente:\.|Ponente:)([\s\S]*?)(?:.###|. Secretar|.Secretar|,|.\nSecretar|\. secretari|\. Relacionada|\. Disidente|\. Improcedencia|\. Semanario|\. Encargad|\. Competencia|\. Incidente|\. Queja|\. Reclamaci|\. Recurso|\. Juicio|\. Jurisprudencia| Magistrado|\. Tomo|\. Vol\wmen|;|\. Nota|\. NOTA|\. En| en funciones| en sustitución|\.; en su| y el engrose|Pág\.|Vol|\.Tom|\. V\wase|\. V\wanse|\. Revisión|\. Amparo| Amparo directo|\. Ausente|\. Séptima|\. Sexta|\.\"\.| \(en su ausencia|\. Texto|\. Cinco|\. Impedido|\. en su ausencia|\. Relator)",text_aux) 
        x1_ponente = re.findall("publicación ([\s\S]*?) del ponente",text_aux )
        x2_ponente = re.findall("Relator: ([\s\S]*?)\.",text_aux ) #re.findall(r"Relator",text_aux)
        x3_ponente = 1
        ponentes = funcion_auxiliar_ponentes(x0_ponente,x1_ponente,x2_ponente,x3_ponente)
        
        if len(re.findall('La publicación no menciona el nombre del ponente',text_aux)) != 0: 
            NA_ponentes = [('NA') for i in range(len(re.findall('La publicación no menciona el nombre del ponente',text_aux)))]
            ponentes.extend(NA_ponentes)
        if len(x2_ponente) != 0: 
            ponentes.extend(x2_ponente)

        #Búsqueda de Fecha
        #fecha_ponente = re.findall(r'\d{1,2} de [a-z]*? de \d{4}', ponente_fecha_texto)
        #fecha = re.findall('(\d{1,2} de [a-z]*?( de)*? \d{4})',ponente_fecha_texto)
        fecha = re.findall('(\d{1,2}( de|de)*? [a-zA-Z]{4,10}( de| del)*? \d{4})',ponente_fecha_texto)
        fecha_aux = pd.DataFrame(fecha,columns=['fecha','del1','del2']).drop(columns=['del1','del2'])
        fecha = fecha_aux['fecha'].values.tolist()
        if len(re.findall('La publicación no menciona la fecha de resolución',text_aux)) != 0: 
            NA_fecha1 = [('NA') for i in range(len(re.findall('La publicación no menciona la fecha de resolución',text_aux)))]
            fecha.extend(NA_fecha1)
    
        if len(fecha) != len(ponentes): 
            NA_fecha2 = [('NA') for i in range(len(ponentes)-len(fecha))]
            fecha.extend(NA_fecha2)
                
        #Búsqueda de Secretario
        #x0_secretario = re.findall("(?:Secretari\w: |Secretari\ws: |Secretari\w. )([\s\S]*?)(\.| Amparo)",text_aux)
        #x0_aux = pd.DataFrame(x0_secretario,columns=['secretario','delete']).drop(columns=['delete'])
        #x0_secretario = x0_aux['secretario'].values.tolist()
        #x1_secretario = re.findall("(?:La publicación |La publicacion)([\s\S]*?) (?:del ponente|el ponente)",text_aux)
        ##x10_secretario = 10
        ##secretarios = funcion_auxiliar_secretarios(x0_secretario,x1_secretario) #,x10_secretario
        #secretarios = x0_secretario
        
        x0_secretario = re.findall("(?:Secretar\w\w: |Secretari\ws: |Secretari\w\. |secretari\w: )([\s\S]*?)(?:\.###|\. Amparo|\. Facultad| Amparo directo|\. Inconformidad|\. Acción|\. Varios|\. Controversia|\. Reconocimiento|\. Expediente vario|\. Consulta|\. Reposici\wn|\. \(|\. La |\. Incompetencia|\. Vol\wmenes|se encargo|Quinta Epoca:|\. Hay cosa|\. Acumulación|Impedimento|Antecedente:|Revisión|Expediente|Los señores|Jurisprudencia|demás que las|XIV.|las demás que|fallado|no superan|Aplicada|De conformidad|contra el|para el|Lo resolvió|Secretarrio:|Presidente|Secretario encargado|ha estimado|Disidente.|Veáse|\. Impedido:|Vease:|Por |Sexta Epoca|Integró|Para|Tribunal|\. Ausente|votos|_________________|Texto|Secretarios|Reproduce|Véanse:|Ponente:|EL|Tomo|Resuelto|Ausentes|Que |Varios|Con |\. Aclaración|Tomando|\. Juicio|Relator|\. Ministro|Incidente|/.|Texto aprobado|Relacionada|Séptima|Acción|Facultad|Controversia|\. Solicitud|Sostiene|La |\. Inejecución|Octava|En |Este |Se |\. Repetición|Reitera|\. El |Contradicción|La presente|El Tribunal|Tesis|La Primera|Sostienen|Informe|Véase|Esta|Importa|Disidentes:|Voto|Ausente:|Secretario:|###|Inconformidad|\. Conflicto|Nota|Criterios|Competencia|Precedente|\. Recurso| Recurso de|Secretaria|;|\. Nota\:|\. Notas\:|Nota\:|Pág\.|Vol|\. Volumen|\. Volúme|\.Tom|\. Véase|\. \. Amparo|\. Amparo di|\.   Amparo dir|\. Amparo in|Amparo en|\. Undécima Época|\. Décima Época|\. Novena Época|\. Octava Época|\. Séptima Época|\. Sexta Época|\. Quinta Época|\. Cuarta Época|\. Tercera Época|\. Segunda Época|\. Primera Época|\. Undécima Epoca|\. Décima Epoca|\. Novena Epoca|\. Octava Epoca|\. Séptima Epoca|\. Sexta Epoca|\. Quinta Epoca|\. Cuarta Epoca|\. Tercera Epoca|\. Segunda Epoca|\. Primera Epoca|\. Improcedencia civil|Sostiene la misma tesis:|\. Revisión fiscal|\. Unanimidad|\. Queja|Queja|\. Competencia|\. Reclamación|\.   Reclamación /.|\.  de enero de|\.  de febrero de|\.  de marzo de|\.  de abril de|\.  de mayo de|\.  de junio de|\.  de julio de|\.  de agosto de|\.  de septiembre de|\.  de octubre de|\.  de noviembre de|\.  de diciembre de|\. Disidente:|\. Precedentes|\. Improcedencia|\. Impedimento|\. Excusa|\. Incidente|\. Encargad|\. Revisión|\. Tomo|por licencia concedida|en funciones de Magistrado por Ministerio de Ley|\. Engrose: |\. Criterios|\. Tesis|\.   Tesis|\. El Tribunal|\. Contradicción de tesis|\. Texto|\. NOTA|\. Recurso de|\.   El Tribunal|\. Véanse|\.   Revisión|.   Notas|\. Semanario|\. Secretario|\. Tercera|\. C\. C\.)", text_aux)                                
        x1_secretario = re.findall("(?:La publicación |La publicacion)([\s\S]*?) (?:del ponente|el ponente)",text_aux)
        x10_secretario = 1
        secretarios = funcion_auxiliar_secretarios(x0_secretario,x1_secretario,x10_secretario)
        if len(secretarios) != len(ponentes):
            NA_sec = [('NA') for i in range(len(ponentes)-len(secretarios))]
            secretarios.extend(NA_sec)
    
        #impresión debug
        print(id_tesis)
        print(fecha)
        print('Ponentes:',ponentes)
        print('Secretarios:',secretarios)

        #Vectores para crear tabla
        vector_id_tesis = len(ponentes)*id_tesis
        vector_reiteracion = [(i+1) for i in range(len(ponentes))]
        vector_tesis = [(tesis) for i in range(len(ponentes))]
        vector_instancia = [(instancia) for i in range(len(ponentes))]
        vector_epoca = [(epoca) for i in range(len(ponentes))]
        vector_materia = [(materia) for i in range(len(ponentes))]

        tabla_iteracion = pd.DataFrame({'id_tesis':vector_id_tesis,'tesis':vector_tesis,'instancia':vector_instancia,'epoca':vector_epoca,'materia':vector_materia,'#_reiteracion':vector_reiteracion,'ponente':ponentes,'secretario':secretarios,'fecha':fecha})
        #tabla_tesis_jurisprudencia = tabla_tesis_jurisprudencia.append(tabla_iteracion)
        
        print(instancia)
        print(epoca)
        print(materia)
        print(tesis)

['815972']
['27 de octubre de 1977', '27 de octubre de 1977', '27 de octubre de 1977', '27 de octubre de 1977', '28 de octubre de 1977']
Ponentes: ['María Cristina Salmorán de Tamayo', 'Julio Sánchez Vargas', 'David Franco Rodríguez', 'María Cristina Salmorán de Tamayo', 'María Cristina Salmorán de Tamayo']
Secretarios: ['Andrés Cruz Martínez', 'Eduardo Aguilar Cota', 'Guillermo Ariza Bracamontes', 'Andrés Cruz Martínez', 'Miguel Bonilla Solís']
Cuarta Sala
Séptima Época
Laboral



In [56]:
tabla_iteracion

,id_tesis,tesis,instancia,epoca,materia,#_reiteracion,ponente,secretario,fecha
0,815972,,Cuarta Sala,Séptima Época,Laboral,1,María Cristina Salmorán de Tamayo,Andrés Cruz Martínez,27 de octubre de 1977
1,815972,,Cuarta Sala,Séptima Época,Laboral,2,Julio Sánchez Vargas,Eduardo Aguilar Cota,27 de octubre de 1977
2,815972,,Cuarta Sala,Séptima Época,Laboral,3,David Franco Rodríguez,Guillermo Ariza Bracamontes,27 de octubre de 1977
3,815972,,Cuarta Sala,Séptima Época,Laboral,4,María Cristina Salmorán de Tamayo,Andrés Cruz Martínez,27 de octubre de 1977
4,815972,,Cuarta Sala,Séptima Época,Laboral,5,María Cristina Salmorán de Tamayo,Miguel Bonilla Solís,28 de octubre de 1977


In [57]:
text_aux

'Tesis Registro digital:  Instancia: Cuarta Sala###Séptima Época###Materia(s): Laboral###Fuente: Semanario Judicial de la Tipo: Jurisprudencia Federación. Volumen 151-156, Quinta Parte, página 163###PREFERENCIA, DERECHO DE. TERMINO DE PRESCRIPCION DE LA ACCION RESPECTIVA.###El artículo 157 de la Ley Federal del Trabajo otorga a los trabajadores que se consideran postergados en sus derechos de preferencia derivados de los artículos 154 y 156 del mismo ordenamiento, dos acciones ejercitables a elección del trabajador afectado, la del otorgamiento del puesto reclamado o la de indemnización consistente en el importe de tres meses de los salarios que correspondan al puesto. En cualquiera de los dos casos, la procedencia de la acción intentada, cualquiera que ella sea, da derecho, además, al pago, por concepto de daños y perjuicios, de los salarios dejados de percibir desde la fecha de la postergación hasta la del cumplimiento del laudo condenatorio. Estas acciones guardan una estrecha analo

In [58]:
ponente_fecha_aux

[(' directo 1635/76. Sección 30 del Sindicato de Trabajadores Petroleros y otro. 27 de octubre de 1977. Cinco votos. ',
  'Ponente: ',
  '',
  ''),
 (' directo 3303/76. Sección 30 del Sindicato de Trabajadores Petroleros de la República Mexicana y otro. 27 de octubre de 1977. Cinco votos. ',
  'Ponente: ',
  '',
  ''),
 (' directo 4320/75. Sección 24 del Sindicato de Trabajadores Petroleros y otros. 27 de octubre de 1977. Cinco votos. ',
  'Ponente: ',
  '',
  ''),
 (' directo 6468/76. Sección 24 del Sindicato de Trabajadores Petroleros de la República Mexicana y otro. 27 de octubre de 1977. Cinco votos. ',
  'Ponente: ',
  '',
  ''),
 (' directo 5057/75. Sección 24 del Sindicato de Trabajadores Petroleros de la República Mexicana. 28 de octubre de 1977. Cinco votos. ',
  'Ponente:',
  '',
  '')]